In [1]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
import asyncio
import nest_asyncio
import sys

# Fix for Windows + Jupyter + Playwright
if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/",
]

config = CrawlerRunConfig(
    cache_mode=CacheMode.BYPASS,
    page_timeout=80000,
    word_count_threshold=50,
)

nest_asyncio.apply()


async def crawl_website():
    data_res = {"data": []}

    async with AsyncWebCrawler() as crawler:
        results = await crawler.arun_many(urls_to_crawl, config=config)

        for result in results:
            if result.success:
                title = result.metadata.get("title", "")
                if not title and result.markdown:
                    lines = result.markdown.raw_markdown.split("\n")
                    for line in lines:
                        if line.startswith("#"):
                            title = line.strip("#").strip()
                            break

                data_res["data"].append(
                    {
                        "text": result.markdown.raw_markdown if result.markdown else "",
                        "meta": {"url": result.url, "meta": {"title": title}},
                    }
                )

    return data_res


data_res = await crawl_website()


NotImplementedError: 